<a href="https://colab.research.google.com/github/nvshubhsharma/Testing-huggingface-models-on-custom-data-/blob/main/Vakyansh_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture

!pip install git+https://github.com/huggingface/transformers.git datasets librosa torch==1.7.0 torchaudio==0.7.0 jiwer

In [ ]:
import torch
import torchaudio
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import re

test_dataset = load_dataset("common_voice", "hi", split="test")
wer = load_metric("wer")

processor = Wav2Vec2Processor.from_pretrained("Harveenchadha/vakyansh-wav2vec2-hindi-him-4200")
model = Wav2Vec2ForCTC.from_pretrained("Harveenchadha/vakyansh-wav2vec2-hindi-him-4200")
model.to("cuda")

/usr/local/lib/python3.7/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


Downloading:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

...

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (5): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        )
        (6): Wav2Vec

In [ ]:


resampler = torchaudio.transforms.Resample(48_000, 16_000)

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“]'

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
  batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
  speech_array, sampling_rate = torchaudio.load(batch["path"])
  batch["speech"] = resampler(speech_array).squeeze().numpy()
  return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def evaluate(batch):
  inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

  with torch.no_grad():
      logits = model(inputs.input_values.to("cuda")).logits

      pred_ids = torch.argmax(logits, dim=-1)
      batch["pred_strings"] = processor.batch_decode(pred_ids, skip_special_tokens=True)
      return batch

result = test_dataset.map(evaluate, batched=True, batch_size=8)

print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))

  0%|          | 0/127 [00:00<?, ?ex/s]

  0%|          | 0/16 [00:00<?, ?ba/s]

WER: 32.946636


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import librosa
import soundfile as sf
import numpy as np

In [ ]:
counter

42

In [ ]:
b=[]
inputs = processor(a, sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
    logits = model(inputs.input_values.to("cuda")).logits
    pred_ids = torch.argmax(logits, dim=-1)
    b = processor.batch_decode(pred_ids, skip_special_tokens=True)
    

In [ ]:
len(a[1])

36750

In [ ]:
a=[]
for i in range (1,42):
  path='/content/drive/MyDrive/'+str(i)+'.wav'
  speech_array, sampling_rate = torchaudio.load(path)
  x =speech_array[0].numpy()
  x=librosa.resample(np.asarray(x), sampling_rate, 16_000)

  a.append(x)


In [ ]:
pred_strings=[]
inputs = processor1(a, sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
    logits = model1(inputs.input_values.to("cuda")).logits

    pred_ids = torch.argmax(logits, dim=-1)
    pred_strings= processor1.batch_decode(pred_ids, skip_special_tokens=True)


In [ ]:
" ".join(pred_strings)

'थैंक यू सो मच मैम दोस्तों कहते हैं कि खूबसूरत से एक पल का किस्सा बन जाता है खूबसूरत से एकपल का किस्सा बन जाता है जाने कब कौन जिंदगी का हिस्सा बन जाता है कुछ खास लोग मिलते हैं जिंदगी में कु कखास लोग मिलते हैं जिंदगी में जिनसे कभी न बिछड़ने वाला रिश्ता बन जाता है आदुर अभिनंदन आभार नमस्कार आदाप स्ट काल हेलो वेरी गुड इवनिंग लेडीज एं जेनमैन वेरीवेरीवॉमरक पसलयूटली वंडरफुल ब्यूटिफुल स्पेश डे् ेशल पीपल लाइक यू आईएम जू ही मल्होत्रा एंड टुडे आई हैव प्लेजर ऑफ बींग होस्टस्पदस्टे देस्तूं आज की इस प्रोग्राम की शुरुआत में करना चाहूंगी सबसे पहले आपका फिर से पुनःमें स्वागत करना चाहूंगी विश्व का सबसे बड़ा राजभाषा हिंदी में तकनीकी सम्मेलन में जहां पर मिलकर इस खूबसूरत्सू उत्सब को या कहें महा सबको हम सेलिब्रेट कर रहे हैं नूस में शुक्रिया अदा करना चाहूंगी आभार व्यक्त करना चाता होंगी श्री सुरेश कुमार जंदरल जी का जो कि डायरेक्टर है रेसिडॉक के और साथ ही साथ में शुक्रिया करना चाहूंगी डॉक्र राजीव विद जी का व्य ज्ञानिक एफ इन मास्से जिनकी मेहनत का नतीजा आज आपके सामने इस प्रोग्राम के माध्यम से हम पहं चा रहे मैं चा

In [ ]:
processor1 = Wav2Vec2Processor.from_pretrained("Harveenchadha/vakyansh-wav2vec2-indian-english-enm-700")
model1 = Wav2Vec2ForCTC.from_pretrained("Harveenchadha/vakyansh-wav2vec2-indian-english-enm-700")
model1.to("cuda")

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (5): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        )
        (6): Wav2Vec

In [ ]:
" ".join(pred_strings)

'thank you so much mam this is now kah tei hain ki kob surat se eqpal ka kesa bhan jata hai khub surat se epal ka kisa bhan jata hai johni kab kan zindagika hisaban jata hai kuch khast log mirte hai zindagi me hch has log milte hai zindagi me jyn se kabhina bichar ne wala rish bhnjata hai adar abhinandan abhar namaskar adab sast a cal helo in a very god evening ladies in gentleman a very very warm welcome to this absolutely wonderful beautiful and ospicious day that brings together so many special people like you i am zuhi malhotra and today i have the pleasure of being a hosters for this day ustu aj ke is program ki shuruat may karna chahungi subsay pahley ap kapret se puna ha mi wagat karna chahungi vishwaka sabsi bata raj bhasha hindi tachniky samailen mai jahpar milker iscup srut suot sabko yah kahi mahab sabko hum celebrate kahrahehai mestumi shukriya adha karna chahungi abharvak karna c ongy sre suresh kumar jendal jika joki director hai residoke ar sat hi sat mashukriya karna ch